In [ ]:
!pip install keras
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from keras.utils import np_utils

iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df[::20]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
20,5.4,3.4,1.7,0.2,0
40,5.0,3.5,1.3,0.3,0
60,5.0,2.0,3.5,1.0,1
80,5.5,2.4,3.8,1.1,1
100,6.3,3.3,6.0,2.5,2
120,6.9,3.2,5.7,2.3,2
140,6.7,3.1,5.6,2.4,2


In [3]:
# 必要なデータを抽出
X = df.drop(['target'], axis=1)
Y = df['target']

In [4]:
# dfをarrayに変換
X_array = np.array(X)
Y_array = np.array(Y)

In [5]:
# [0, 1, 2]を多クラスに分ける（0なら[1, 0, 0], 1なら[0, 1, 0], ...）
Y_array = np_utils.to_categorical(Y_array)

In [6]:
# データ確認（20づつ表示）
Y_array[::20]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split

# 学習データと検証データに分ける
train_x, test_x, train_y, test_y = train_test_split(X_array, Y_array, train_size=0.8, test_size=0.2, random_state=144)
len(train_x), len(test_x), len(train_y), len(test_y) # サイズ表示

(120, 30, 120, 30)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(input_dim=4, units=3)) # 入力が4種の特徴、3つに分離
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.1))
model.fit(train_x, train_y, epochs=100, batch_size=10) # エポック100, ミニバッチ 10

Epoch 1/100
12/12 [==============================] - 17s 2ms/step - loss: 1.2517
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 0.8577
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 0.9813
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 0.5806
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 0.8551
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 0.4908
Epoch 7/100
12/12 [==============================] - 0s 4ms/step - loss: 0.4570
Epoch 8/100
12/12 [==============================] - 0s 7ms/step - loss: 1.1808
Epoch 9/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4483
Epoch 10/100
12/12 [==============================] - 0s 3ms/step - loss: 0.3912
Epoch 11/100
12/12 [==============================] - 0s 3ms/step - loss: 0.4578
Epoch 12/100
12/12 [==============================] - 0s 3ms/step - loss: 0.3836
Epoch 13/100
12/12 [================

In [9]:
_, predict_y = np.where(0 < (model.predict(test_x, batch_size=10) > 0.5).astype('int32'))
predict_y

array([1, 2, 0, 2, 2, 0, 1, 2, 2, 0, 1, 1, 2, 0, 1, 0, 2, 1, 1, 2, 0, 0,
       1, 1, 1, 2, 2, 1, 2, 1])

In [10]:
# 正解データの取得
_, Y_index = np.where(test_y > 0)
print(Y_index)

[1 2 0 2 2 0 1 2 2 0 1 1 2 0 1 0 2 1 1 2 0 0 1 1 1 2 2 1 2 1]


In [11]:
print('True:', np.count_nonzero(predict_y == Y_index))
print('False:', np.count_nonzero(predict_y != Y_index))

True: 30
False: 0


In [12]:
from sklearn.metrics import classification_report

print(classification_report(Y_index, predict_y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

